In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split

In [ ]:
!pip install smote-variants

In [2]:
import smote_variants as sv

# Helper Functions

In [4]:
def err_metric(CM): 
      
    TN = CM.iloc[0,0]
    FN = CM.iloc[1,0]
    TP = CM.iloc[1,1]
    FP = CM.iloc[0,1]
    precision =(TP)/(TP+FP)
    accuracy_model  =(TP+TN)/(TP+TN+FP+FN)
    recall_score  =(TP)/(TP+FN)
    specificity_value =(TN)/(TN + FP)
      
    False_positive_rate =(FP)/(FP+TN)
    False_negative_rate =(FN)/(FN+TP)
    f1_score =2*(( precision * recall_score)/( precision + recall_score))
    print("Precision value of the model: ",precision)
    print("Accuracy of the model: ",accuracy_model)
    print("Recall of the model: ",recall_score)
    print("Specificity of the model: ",specificity_value)

# Data

In [27]:
df=pd.read_csv('/content/delirium_dataset_catboost_encoded_and_scaled_55.csv')
df.drop('Unnamed: 0',inplace=True,axis=1)

In [ ]:
delirium_dataset_ohe_encoded=pd.read_csv('/content/delirium_dataset_encoded.csv')
delirium_dataset_ohe_encoded.drop('Unnamed: 0',inplace=True,axis=1)

In [ ]:
delirium_dataset_sohe_encoded=pd.read_csv('/content/delirium_dataset_encoded_and_scaled.csv')
delirium_dataset_sohe_encoded.drop('Unnamed: 0',inplace=True,axis=1)

In [ ]:
delirium_dataset_catboost_encoded=pd.read_csv('/content/delirium_dataset_catboost_encoded.csv')
delirium_dataset_catboost_encoded.drop('Unnamed: 0',inplace=True,axis=1)

In [ ]:
df=pd.read_csv('/content/delirium_dataset_catboost_encoded_and_scaled.csv')
df.drop('Unnamed: 0',inplace=True,axis=1)

In [28]:
df

,ADMISSION_LOCATION,ADMISSION_TYPE,AGE,ALT,ART BP Systolic,AST,Admission Weight (Kg),Alkaline Phosphate,Anion gap,Arterial Base Excess,...,Non Invasive Blood Pressure mean,Non-Invasive Blood Pressure mean,O2 saturation pulseoxymetry,Potassium (whole blood),RELIGION,Respiratory Rate,Sodium (serum),Temperature Celsius,Total Bilirubin,WBC
0,0.126311,0.097822,0.195363,0.766596,0.025021,0.324367,0.183374,0.485249,0.083333,0.851319,...,0.048792,0.086530,0.005570,0.053642,0.132305,0.116003,0.580037,0.366055,0.417135,0.030160
1,0.126311,0.097822,0.030938,0.766690,0.036163,0.326883,0.084803,0.498293,0.170238,0.755870,...,0.075080,0.088240,0.006374,0.071472,0.061742,0.111528,0.424598,0.359693,0.412661,0.012610
2,0.708770,0.699274,0.029788,0.768550,0.041572,0.332079,0.092490,0.494139,0.189286,0.777897,...,0.083632,0.089750,0.006264,0.078445,0.470905,0.144941,0.520707,0.358907,0.415345,0.010125
3,0.902923,0.899758,0.030706,0.767430,0.035072,0.328746,0.085372,0.503976,0.189286,0.763212,...,0.067794,0.087195,0.006080,0.081693,0.675487,0.104405,0.412517,0.359906,0.414749,0.012071
4,0.126311,1.000000,0.203450,0.769025,0.023423,0.330162,0.132627,0.490568,0.140306,0.778913,...,0.022947,0.086659,0.006262,0.065413,0.132305,0.124731,0.484363,0.374433,0.423781,0.022021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8641,0.139874,0.104485,0.293799,0.761841,0.034161,0.328157,0.080137,0.497751,0.144286,0.825647,...,0.075117,0.087291,0.006382,0.095613,0.117617,0.110008,0.443927,0.356144,0.421907,0.010450
8642,0.140159,0.104645,0.236879,0.772527,0.018894,0.332733,0.123726,0.496697,0.084286,0.828333,...,0.040732,0.086606,0.006347,0.056299,0.143258,0.097623,0.498371,0.367473,0.421740,0.019659
8643,0.140641,0.104624,0.231410,0.771572,0.017983,0.332924,0.129137,0.498297,0.171429,0.831740,...,0.023801,0.086720,0.005580,0.056004,0.143211,0.075177,0.521405,0.349049,0.469733,0.020472
8644,0.140120,0.104604,0.225473,0.766568,0.024141,0.323942,0.139450,0.496551,0.058333,0.857744,...,0.036900,0.088543,0.006119,0.064469,0.118213,0.139349,0.578292,0.361831,0.428223,0.019960



# Oversampling Tuning

In [7]:
import smote_variants as sv
import imblearn.datasets as imb_datasets

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [29]:
ovs='CBSO'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 19:33:40,698:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 19:33:40,729:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 19:33:40,745:INFO:CBSO: Running sampling via ('CBSO', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'C_p': 1.3, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'CBSO'}")
INFO:smote_variants:CBSO: Running sampling via ('CBSO', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'C_p': 1.3, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, '

model
RandomForestClassifier()
confusion matrix
[[2120  169]
 [ 170 2165]]
classification report
              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93      2289
         1.0       0.93      0.93      0.93      2335

    accuracy                           0.93      4624
   macro avg       0.93      0.93      0.93      4624
weighted avg       0.93      0.93      0.93      4624

Accuracy : 0.926687
Area under the curve : 0.926682


In [10]:
ovs='DBSMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:17:54,418:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:17:54,427:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:17:54,440:INFO:DBSMOTE: Running sampling via ('DBSMOTE', "{'proportion': 1.0, 'eps': 0.8, 'min_samples': 3, 'db_iter_limit': 3, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'DBSMOTE'}")
INFO:smote_variants:DBSMOTE: Running sampling via ('DBSMOTE', "{'proportion': 1.0, 'eps': 0.8, 'min_samples': 3, 'db_iter_limit': 3, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'DBSMOTE'}")
2022-08-22 17:17:54,511:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
INFO:smote_variants:NearestNeighborsWithMetricTensor: NN fittin

model
RandomForestClassifier()
confusion matrix
[[2289    0]
 [ 262 2073]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95      2289
         1.0       1.00      0.89      0.94      2335

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.943339
Area under the curve : 0.943897


In [11]:
ovs='ASMOBD'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:18:09,830:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:18:09,839:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:18:09,850:INFO:ASMOBD: Running sampling via ('ASMOBD', "{'proportion': 1.0, 'min_samples': 3, 'eps': 0.8, 'eta': 0.5, 'T_12': (1.0, 1.0), 't_12': (4.0, 4.0), 'a': 0.05, 'smoothing': 'linear', 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'ASMOBD'}")
INFO:smote_variants:ASMOBD: Running sampling via ('ASMOBD', "{'proportion': 1.0, 'min_samples': 3, 'eps': 0.8, 'eta': 0.5, 'T_12': (1.0, 1.0), 't_12': (4.0, 4.0), 'a': 0.05, 'smoothing'

model
RandomForestClassifier()
confusion matrix
[[2288    1]
 [ 262 2073]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95      2289
         1.0       1.00      0.89      0.94      2335

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.943123
Area under the curve : 0.943679


In [12]:
ovs='Assembled_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:20:09,078:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:20:09,089:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:20:09,102:INFO:Assembled_SMOTE: Running sampling via ('Assembled_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'pop': 2, 'thres': 0.3, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'Assembled_SMOTE'}")
INFO:smote_variants:Assembled_SMOTE: Running sampling via ('Assembled_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'pop': 2, 'thres': 0.3, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'with

model
RandomForestClassifier()
confusion matrix
[[2178  111]
 [ 126 2209]]
classification report
              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95      2289
         1.0       0.95      0.95      0.95      2335

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.948746
Area under the curve : 0.948773


In [13]:
ovs='SDSMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:20:39,995:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:20:40,005:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:20:40,018:INFO:SDSMOTE: Running sampling via ('SDSMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'SDSMOTE'}")
INFO:smote_variants:SDSMOTE: Running sampling via ('SDSMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_nam

model
RandomForestClassifier()
confusion matrix
[[2179  110]
 [ 127 2208]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.95      2289
         1.0       0.95      0.95      0.95      2335

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.948746
Area under the curve : 0.948777


In [15]:
ovs='G_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:28:40,964:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:28:40,977:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:28:40,991:INFO:G_SMOTE: Running sampling via ('G_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'method': 'linear', 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'G_SMOTE'}")
INFO:smote_variants:G_SMOTE: Running sampling via ('G_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'method': 'linear', 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_compon

model
RandomForestClassifier()
confusion matrix
[[2241   48]
 [ 145 2190]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96      2289
         1.0       0.98      0.94      0.96      2335

    accuracy                           0.96      4624
   macro avg       0.96      0.96      0.96      4624
weighted avg       0.96      0.96      0.96      4624

Accuracy : 0.958261
Area under the curve : 0.958466


In [16]:
ovs='NT_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:28:49,592:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:28:49,603:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:28:49,617:INFO:NT_SMOTE: Running sampling via ('NT_SMOTE', "{'proportion': 1.0, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 3, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'NT_SMOTE'}")
INFO:smote_variants:NT_SMOTE: Running sampling via ('NT_SMOTE', "{'proportion': 1.0, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 3, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'NT_SMOTE'}")
2022-08-22 17

model
RandomForestClassifier()
confusion matrix
[[2250   39]
 [ 225 2110]]
classification report
              precision    recall  f1-score   support

         0.0       0.91      0.98      0.94      2289
         1.0       0.98      0.90      0.94      2335

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.942907
Area under the curve : 0.943301


In [17]:
ovs='Lee'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:29:01,702:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:29:01,714:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:29:01,727:INFO:Lee: Running sampling via ('Lee', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'rejection_level': 0.5, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'Lee'}")
INFO:smote_variants:Lee: Running sampling via ('Lee', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'rejection_level': 0.5, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, '

model
RandomForestClassifier()
confusion matrix
[[2255   34]
 [ 187 2148]]
classification report
              precision    recall  f1-score   support

         0.0       0.92      0.99      0.95      2289
         1.0       0.98      0.92      0.95      2335

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.952206
Area under the curve : 0.952530


In [20]:
ovs='MDO'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:30:23,153:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:30:23,162:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:30:23,175:INFO:MDO: Running sampling via ('MDO', "{'proportion': 1.0, 'K2': 5, 'K1_frac': 0.5, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'MDO'}")
INFO:smote_variants:MDO: Running sampling via ('MDO', "{'proportion': 1.0, 'K2': 5, 'K1_frac': 0.5, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'MDO'}")
2022-08-22 17:30:23,181:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
INFO:smote_variants:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-22 17:30:23,186:INFO:NearestNeighborsWithM

model
RandomForestClassifier()
confusion matrix
[[2289    0]
 [ 262 2073]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95      2289
         1.0       1.00      0.89      0.94      2335

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.943339
Area under the curve : 0.943897


In [21]:
ovs='Random_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:30:32,632:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:30:32,643:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:30:32,657:INFO:Random_SMOTE: Running sampling via ('Random_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'Random_SMOTE'}")
INFO:smote_variants:Random_SMOTE: Running sampling via ('Random_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'Random_SMOTE'}")
2022-08-22 17:30:32,666:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
INFO:smote_variants:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-22

model
RandomForestClassifier()
confusion matrix
[[2187  102]
 [ 199 2136]]
classification report
              precision    recall  f1-score   support

         0.0       0.92      0.96      0.94      2289
         1.0       0.95      0.91      0.93      2335

    accuracy                           0.93      4624
   macro avg       0.94      0.94      0.93      4624
weighted avg       0.94      0.93      0.93      4624

Accuracy : 0.934905
Area under the curve : 0.935107


In [22]:
ovs='VIS_RST'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:30:36,947:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:30:36,958:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:30:36,972:INFO:VIS_RST: Running sampling via ('VIS_RST', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'VIS_RST'}")
INFO:smote_variants:VIS_RST: Running sampling via ('VIS_RST', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_nam

model
RandomForestClassifier()
confusion matrix
[[2141  158]
 [ 391  559]]
classification report
              precision    recall  f1-score   support

         0.0       0.85      0.93      0.89      2299
         1.0       0.78      0.59      0.67       950

    accuracy                           0.83      3249
   macro avg       0.81      0.76      0.78      3249
weighted avg       0.83      0.83      0.82      3249

Accuracy : 0.831025
Area under the curve : 0.759848


In [23]:
ovs='A_SUWO'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:30:45,943:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:30:45,954:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:30:45,967:INFO:A_SUWO: Running sampling via ('A_SUWO', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_clus_maj': 7, 'c_thres': 0.8, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'A_SUWO'}")
INFO:smote_variants:A_SUWO: Running sampling via ('A_SUWO', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_clus_maj': 7, 'c_thres': 0.8, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'rand

model
RandomForestClassifier()
confusion matrix
[[2311    0]
 [ 196 2009]]
classification report
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      2311
         1.0       1.00      0.91      0.95      2205

    accuracy                           0.96      4516
   macro avg       0.96      0.96      0.96      4516
weighted avg       0.96      0.96      0.96      4516

Accuracy : 0.956599
Area under the curve : 0.955556


In [25]:
ovs='AND_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:42:29,155:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:42:29,165:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:42:29,180:INFO:AND_SMOTE: Running sampling via ('AND_SMOTE', "{'proportion': 1.0, 'K': 15, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'AND_SMOTE'}")
INFO:smote_variants:AND_SMOTE: Running sampling via ('AND_SMOTE', "{'proportion': 1.0, 'K': 15, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'AND

model
RandomForestClassifier()
confusion matrix
[[2246   43]
 [ 127 2208]]
classification report
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.96      2289
         1.0       0.98      0.95      0.96      2335

    accuracy                           0.96      4624
   macro avg       0.96      0.96      0.96      4624
weighted avg       0.96      0.96      0.96      4624

Accuracy : 0.963235
Area under the curve : 0.963412


In [26]:
ovs='NRAS'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:42:41,951:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:42:41,966:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:42:41,978:INFO:NRAS: Running sampling via ('NRAS', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 't': 0.5, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'NRAS'}")
INFO:smote_variants:NRAS: Running sampling via ('NRAS', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 't': 0.5, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'clas

model
RandomForestClassifier()
confusion matrix
[[2260   29]
 [ 219 2116]]
classification report
              precision    recall  f1-score   support

         0.0       0.91      0.99      0.95      2289
         1.0       0.99      0.91      0.94      2335

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.946367
Area under the curve : 0.946770


In [28]:
ovs='SSO'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:43:05,949:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:43:05,960:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:43:05,971:INFO:SSO: Running sampling via ('SSO', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'h': 10, 'n_iter': 5, 'n_jobs': 1, 'random_state': 5, 'class_name': 'SSO'}")
INFO:smote_variants:SSO: Running sampling via ('SSO', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'h': 10, 'n_iter': 5, 'n_jobs': 1, 'random_state': 5, 'class_name': 'SSO'}")
2022-08-22 17:43:07,792:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
INFO:smote_variants:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-22 17:43:07,

model
RandomForestClassifier()
confusion matrix
[[2289    0]
 [ 264 2071]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95      2289
         1.0       1.00      0.89      0.94      2335

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.942907
Area under the curve : 0.943469


In [29]:
ovs='NDO_sampling'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:43:41,776:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:43:41,800:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:43:41,834:INFO:NDO_sampling: Running sampling via ('NDO_sampling', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': None}, 'T': 0.5, 'n_jobs': 1, 'random_state': 5, 'class_name': 'NDO_sampling'}")
INFO:smote_variants:NDO_sampling: Running sampling via ('NDO_sampling', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': None}, 'T'

model
RandomForestClassifier()
confusion matrix
[[2178  111]
 [ 143 2192]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.94      2289
         1.0       0.95      0.94      0.95      2335

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.945069
Area under the curve : 0.945133


In [31]:
ovs='Gaussian_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:47:08,298:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:47:08,310:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:47:08,322:INFO:Gaussian_SMOTE: Running sampling via ('Gaussian_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'sigma': 1.0, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {'sigma': 1.0}}, 'random_state': 5, 'class_name': 'Gaussian_SMOTE'}")
INFO:smote_variants:Gaussian_SMOTE: Running sampling via ('Gaussian_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'sigma': 1.0, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sa

model
RandomForestClassifier()
confusion matrix
[[2289    0]
 [ 264 2071]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95      2289
         1.0       1.00      0.89      0.94      2335

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.942907
Area under the curve : 0.943469


In [32]:
ovs='kmeans_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:47:12,535:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:47:12,546:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:47:12,559:INFO:kmeans_SMOTE: Running sampling via ('kmeans_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_clusters': 10, 'irt': 2.0, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'kmeans_SMOTE'}")
INFO:smote_variants:kmeans_SMOTE: Running sampling via ('kmeans_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_clusters': 10, 'irt': 2.0, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_

model
RandomForestClassifier()
confusion matrix
[[2303    0]
 [ 283    8]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      2303
         1.0       1.00      0.03      0.05       291

    accuracy                           0.89      2594
   macro avg       0.95      0.51      0.50      2594
weighted avg       0.90      0.89      0.84      2594

Accuracy : 0.890902
Area under the curve : 0.513746


In [4]:
ovs='CCR'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 18:32:38,266:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 18:32:38,295:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 18:32:38,313:INFO:CCR: Running sampling via ('CCR', "{'proportion': 1.0, 'energy': 1.0, 'scaling': 0.0, 'n_jobs': 1, 'random_state': 5, 'class_name': 'CCR'}")
INFO:smote_variants:CCR: Running sampling via ('CCR', "{'proportion': 1.0, 'energy': 1.0, 'scaling': 0.0, 'n_jobs': 1, 'random_state': 5, 'class_name': 'CCR'}")


model
RandomForestClassifier()
confusion matrix
[[2313    0]
 [ 255 2208]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95      2313
         1.0       1.00      0.90      0.95      2463

    accuracy                           0.95      4776
   macro avg       0.95      0.95      0.95      4776
weighted avg       0.95      0.95      0.95      4776

Accuracy : 0.946608
Area under the curve : 0.948234


In [5]:
ovs='ANS'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 18:32:52,300:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 18:32:52,319:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 18:32:52,335:INFO:ANS: Running sampling via ('ANS', "{'proportion': 1.0, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'ANS'}")
INFO:smote_variants:ANS: Running sampling via ('ANS', "{'proportion': 1.0, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'ANS'}")
2022-08-22 18:32:52,348:INFO:NearestNeighbo

model
RandomForestClassifier()
confusion matrix
[[2251   38]
 [ 185 2150]]
classification report
              precision    recall  f1-score   support

         0.0       0.92      0.98      0.95      2289
         1.0       0.98      0.92      0.95      2335

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.951773
Area under the curve : 0.952085


In [6]:
ovs='cluster_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 18:34:11,035:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 18:34:11,044:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 18:34:11,054:INFO:cluster_SMOTE: Running sampling via ('cluster_SMOTE', "{'proportion': 1.0, 'n_neighbors': 3, 'nn_params': {}, 'n_clusters': 3, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'cluster_SMOTE'}")
INFO:smote_variants:cluster_SMOTE: Running sampling via ('cluster_SMOTE', "{'proportion': 1.0, 'n_neighbors': 3, 'nn_params': {}, 'n_clusters': 3, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'r

model
RandomForestClassifier()
confusion matrix
[[2234   55]
 [ 144 2191]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96      2289
         1.0       0.98      0.94      0.96      2335

    accuracy                           0.96      4624
   macro avg       0.96      0.96      0.96      4624
weighted avg       0.96      0.96      0.96      4624

Accuracy : 0.956964
Area under the curve : 0.957151


In [7]:
ovs='SYMPROD'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 18:34:20,599:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 18:34:20,608:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 18:34:20,624:INFO:SYMPROD: Running sampling via ('SYMPROD', "{'proportion': 1.0, 'std_outliers': 3, 'k_neighbors': 7, 'm_neighbors': 7, 'cutoff_threshold': 1.25, 'nn_params': {}, 'random_state': 5, 'class_name': 'SYMPROD'}")
INFO:smote_variants:SYMPROD: Running sampling via ('SYMPROD', "{'proportion': 1.0, 'std_outliers': 3, 'k_neighbors': 7, 'm_neighbors': 7, 'cutoff_threshold': 1.25, 'nn_params': {}, 'random_state': 5, 'class_name': 'SYMPROD'}")
2022-08-22 18:34:20,662:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
INFO:smote_variants:Nearest

model
RandomForestClassifier()
confusion matrix
[[2303    0]
 [ 283    8]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      2303
         1.0       1.00      0.03      0.05       291

    accuracy                           0.89      2594
   macro avg       0.95      0.51      0.50      2594
weighted avg       0.90      0.89      0.84      2594

Accuracy : 0.890902
Area under the curve : 0.513746


In [8]:
ovs='Edge_Det_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 18:34:35,814:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 18:34:35,826:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 18:34:35,839:INFO:Edge_Det_SMOTE: Running sampling via ('Edge_Det_SMOTE', "{'proportion': 1.0, 'k': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'Edge_Det_SMOTE'}")
INFO:smote_variants:Edge_Det_SMOTE: Running sampling via ('Edge_Det_SMOTE', "{'proportion': 1.0, 'k': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state'

model
RandomForestClassifier()
confusion matrix
[[2179  110]
 [ 134 2201]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.95      2289
         1.0       0.95      0.94      0.95      2335

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.947232
Area under the curve : 0.947278


In [9]:
ovs='CE_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 18:34:47,128:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 18:34:47,139:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 18:34:47,152:INFO:CE_SMOTE: Running sampling via ('CE_SMOTE', "{'proportion': 1.0, 'h': 10, 'k': 5, 'nn_params': {}, 'alpha': 0.5, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'CE_SMOTE'}")
INFO:smote_variants:CE_SMOTE: Running sampling via ('CE_SMOTE', "{'proportion': 1.0, 'h': 10, 'k': 5, 'nn_params': {}, 'alpha': 0.5, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}

model
RandomForestClassifier()
confusion matrix
[[2200   89]
 [ 117 2218]]
classification report
              precision    recall  f1-score   support

         0.0       0.95      0.96      0.96      2289
         1.0       0.96      0.95      0.96      2335

    accuracy                           0.96      4624
   macro avg       0.96      0.96      0.96      4624
weighted avg       0.96      0.96      0.96      4624

Accuracy : 0.955450
Area under the curve : 0.955506


In [11]:
ovs='SOMO'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 18:37:03,039:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 18:37:03,047:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 18:37:03,057:INFO:SOMO: Running sampling via ('SOMO', "{'proportion': 1.0, 'n_grid': 10, 'sigma': 0.2, 'learning_rate': 0.5, 'n_iter': 100, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'SOMO'}")
INFO:smote_variants:SOMO: Running sampling via ('SOMO', "{'proportion': 1.0, 'n_grid': 10, 'sigma': 0.2, 'learning_rate': 0.5, 'n_iter': 100, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_c

model
RandomForestClassifier()
confusion matrix
[[2303    0]
 [ 284    7]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      2303
         1.0       1.00      0.02      0.05       291

    accuracy                           0.89      2594
   macro avg       0.95      0.51      0.49      2594
weighted avg       0.90      0.89      0.84      2594

Accuracy : 0.890517
Area under the curve : 0.512027


In [13]:
ovs='OUPS'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 18:38:00,896:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 18:38:00,915:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 18:38:00,935:INFO:OUPS: Running sampling via ('OUPS', "{'proportion': 1.0, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'OUPS'}")
INFO:smote_variants:OUPS: Running sampling via ('OUPS', "{'proportion': 1.0, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'OUPS'}")
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.p

model
RandomForestClassifier()
confusion matrix
[[2281    8]
 [ 264 2071]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.94      2289
         1.0       1.00      0.89      0.94      2335

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.941176
Area under the curve : 0.941721


In [33]:
ovs='SMOTE_D'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))


2022-08-22 19:35:45,290:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 19:35:45,302:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 19:35:45,312:INFO:SMOTE_D: Running sampling via ('SMOTE_D', "{'proportion': 1.0, 'k': 3, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'SMOTE_D'}")
INFO:smote_variants:SMOTE_D: Running sampling via ('SMOTE_D', "{'proportion': 1.0, 'k': 3, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'SMOTE_D'}")
2022-08-22 19:35:45,317:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
INFO:smote_variants:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-22 19:35:45,321:INFO:NearestNeighborsWithMetricTenso

model
RandomForestClassifier()
confusion matrix
[[2207   96]
 [ 210 2112]]
classification report
              precision    recall  f1-score   support

         0.0       0.91      0.96      0.94      2303
         1.0       0.96      0.91      0.93      2322

    accuracy                           0.93      4625
   macro avg       0.93      0.93      0.93      4625
weighted avg       0.93      0.93      0.93      4625

Accuracy : 0.933838
Area under the curve : 0.933938


In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2289    0]
 [ 266 2069]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95      2289
         1.0       1.00      0.89      0.94      2335

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.942474
Area under the curve : 0.943041


In [31]:
ovs='V_SYNTH'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 19:35:03,916:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 19:35:03,936:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 19:35:03,947:INFO:V_SYNTH: Running sampling via ('V_SYNTH', "{'proportion': 1.0, 'n_components': 3, 'n_jobs': 1, 'random_state': 5, 'class_name': 'V_SYNTH'}")
INFO:smote_variants:V_SYNTH: Running sampling via ('V_SYNTH', "{'proportion': 1.0, 'n_components': 3, 'n_jobs': 1, 'random_state': 5, 'class_name': 'V_SYNTH'}")


model
RandomForestClassifier()
confusion matrix
[[2289    0]
 [ 269 2066]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      2289
         1.0       1.00      0.88      0.94      2335

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.941825
Area under the curve : 0.942398


In [ ]:
ovs='SMOTE_IPF'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

In [42]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
ovs='MCT'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.30, random_state=0)
clf_xgb = xgb.XGBClassifier(colsample_bytree= 0.6,
          gamma=0.15,
          scale_pos_weight= 30,
          eta= 0.4,
          max_depth= 25,
          min_child_weight=17,
          subsample= 0.9,objective='binary:logistic',
                            eval_metric="logloss", ## this avoids a warning...
                            seed=42, 
                            use_label_encoder=False)
clf_xgb.fit(xtrain, ytrain)

prd=clf_xgb.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, prd))
print('classification report')
print(metrics.classification_report(ytest, prd))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, prd)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, prd)))

confusion matrix
[[2146  157]
 [ 197   94]]
classification report
              precision    recall  f1-score   support

         0.0       0.92      0.93      0.92      2303
         1.0       0.37      0.32      0.35       291

    accuracy                           0.86      2594
   macro avg       0.65      0.63      0.64      2594
weighted avg       0.86      0.86      0.86      2594

Accuracy : 0.863531
Area under the curve : 0.627426


In [41]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
ovs='MCT'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
clf_xgb = xgb.XGBClassifier(colsample_bytree= 0.6,
          gamma=0.15,
          scale_pos_weight= 30,
          eta= 0.4,
          max_depth= 25,
          min_child_weight=17,
          subsample= 0.9,objective='binary:logistic',
                            eval_metric="logloss", ## this avoids a warning...
                            seed=42, 
                            use_label_encoder=False)
clf_xgb.fit(xtrain, ytrain)

prd=clf_xgb.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, prd))
print('classification report')
print(metrics.classification_report(ytest, prd))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, prd)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, prd)))

2022-08-22 20:18:02,061:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 20:18:02,071:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 20:18:02,099:INFO:MCT: Running sampling via ('MCT', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 1, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'MCT'}")
INFO:smote_variants:MCT: Running sampling via ('MCT', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 1, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'MCT'}")
2022-08

confusion matrix
[[1745  544]
 [   5 2330]]
classification report
              precision    recall  f1-score   support

         0.0       1.00      0.76      0.86      2289
         1.0       0.81      1.00      0.89      2335

    accuracy                           0.88      4624
   macro avg       0.90      0.88      0.88      4624
weighted avg       0.90      0.88      0.88      4624

Accuracy : 0.881272
Area under the curve : 0.880100


In [35]:
ovs='MCT'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 19:36:24,184:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 19:36:24,195:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 19:36:24,206:INFO:MCT: Running sampling via ('MCT', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 1, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'MCT'}")
INFO:smote_variants:MCT: Running sampling via ('MCT', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 1, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'MCT'}")
2022-08

model
RandomForestClassifier()
confusion matrix
[[2278   11]
 [  11 2324]]
classification report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2289
         1.0       1.00      1.00      1.00      2335

    accuracy                           1.00      4624
   macro avg       1.00      1.00      1.00      4624
weighted avg       1.00      1.00      1.00      4624

Accuracy : 0.995242
Area under the curve : 0.995242


In [34]:
ovs='Gazzah'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 19:36:09,817:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 19:36:09,844:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 19:36:09,865:INFO:Gazzah: Running sampling via ('Gazzah', "{'proportion': 1.0, 'n_components': 2, 'n_jobs': 1, 'random_state': 5, 'class_name': 'Gazzah'}")
INFO:smote_variants:Gazzah: Running sampling via ('Gazzah', "{'proportion': 1.0, 'n_components': 2, 'n_jobs': 1, 'random_state': 5, 'class_name': 'Gazzah'}")
2022-08-22 19:36:09,872:INFO:polynom_fit_SMOTE_star: Running sampling via ('polynom_fit_SMOTE_star', "{'proportion': 1.0, 'topology': 'star', 'random_state': 5, 'class_name': 'polynom_fit_SMOTE_star'}")
INFO:smote_variants:polynom_fit_SMOTE_star: Running sampli

model
RandomForestClassifier()
confusion matrix
[[2276    1]
 [ 280  670]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      2277
         1.0       1.00      0.71      0.83       950

    accuracy                           0.91      3227
   macro avg       0.94      0.85      0.88      3227
weighted avg       0.92      0.91      0.91      3227

Accuracy : 0.912922
Area under the curve : 0.852412


In [35]:
ovs='DEAGO'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:48:40,795:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:48:40,804:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:48:40,814:INFO:DEAGO: Running sampling via ('DEAGO', "{'proportion': 1.0, 'n_neighbors': 5, 'e': 100, 'h': 0.3, 'sigma': 0.1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': None}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'DEAGO'}")
INFO:smote_variants:DEAGO: Running sampling via ('DEAGO', "{'proportion': 1.0, 'n_neighbors': 5, 'e': 100, 'h': 0.3, 'sigma': 0.1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': None}, 'n_jobs': 1, '

model
RandomForestClassifier()
confusion matrix
[[2289    0]
 [ 264 2071]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95      2289
         1.0       1.00      0.89      0.94      2335

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.942907
Area under the curve : 0.943469


In [36]:
ovs='RWO_sampling'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 19:36:40,499:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 19:36:40,507:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 19:36:40,516:INFO:RWO_sampling: Running sampling via ('RWO_sampling', "{'proportion': 1.0, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 1, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'RWO_sampling'}")
INFO:smote_variants:RWO_sampling: Running sampling via ('RWO_sampling', "{'proportion': 1.0, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 1, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'RWO_sa

model
RandomForestClassifier()
confusion matrix
[[2268   21]
 [ 110 2225]]
classification report
              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97      2289
         1.0       0.99      0.95      0.97      2335

    accuracy                           0.97      4624
   macro avg       0.97      0.97      0.97      4624
weighted avg       0.97      0.97      0.97      4624

Accuracy : 0.971670
Area under the curve : 0.971858


In [37]:
ovs='PDFOS'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))


2022-08-22 19:37:00,958:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 19:37:00,964:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 19:37:00,973:INFO:PDFOS: Running sampling via ('PDFOS', "{'proportion': 1.0, 'n_jobs': 1, 'random_state': 5, 'class_name': 'PDFOS'}")
INFO:smote_variants:PDFOS: Running sampling via ('PDFOS', "{'proportion': 1.0, 'n_jobs': 1, 'random_state': 5, 'class_name': 'PDFOS'}")
2022-08-22 19:37:01,035:INFO:MetricTensor: executing metric learning with cov_min
INFO:smote_variants:MetricTensor: executing metric learning with cov_min


model
RandomForestClassifier()
confusion matrix
[[2275   14]
 [ 260 2075]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      0.99      0.94      2289
         1.0       0.99      0.89      0.94      2335

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.940744
Area under the curve : 0.941267


In [38]:
ovs='MWMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:49:15,038:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:49:15,048:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:49:15,064:INFO:MWMOTE: Running sampling via ('MWMOTE', "{'proportion': 1.0, 'k1': 5, 'k2': 5, 'k3': 5, 'M': 10, 'cf_th': 5.0, 'cmax': 10.0, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'MWMOTE'}")
INFO:smote_variants:MWMOTE: Running sampling via ('MWMOTE', "{'proportion': 1.0, 'k1': 5, 'k2': 5, 'k3': 5, 'M': 10, 'cf_th': 5.0, 'cmax': 10.0, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'withi

model
RandomForestClassifier()
confusion matrix
[[2266   23]
 [ 205 2130]]
classification report
              precision    recall  f1-score   support

         0.0       0.92      0.99      0.95      2289
         1.0       0.99      0.91      0.95      2335

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.950692
Area under the curve : 0.951079


In [38]:
ovs='LN_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 19:37:17,163:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 19:37:17,172:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 19:37:17,179:INFO:LN_SMOTE: Running sampling via ('LN_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'LN_SMOTE'}")
INFO:smote_variants:LN_SMOTE: Running sampling via ('LN_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'LN_SMOTE'}")
2022-08-22 19:37:17,189:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
INFO:smote_variants:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-22 19:37:17,193:INFO:Neare

model
RandomForestClassifier()
confusion matrix
[[2213   76]
 [ 131 2204]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.97      0.96      2289
         1.0       0.97      0.94      0.96      2335

    accuracy                           0.96      4624
   macro avg       0.96      0.96      0.96      4624
weighted avg       0.96      0.96      0.96      4624

Accuracy : 0.955234
Area under the curve : 0.955347


In [39]:
ovs='Selected_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 19:37:32,323:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 19:37:32,333:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 19:37:32,344:INFO:Selected_SMOTE: Running sampling via ('Selected_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'perc_sign_attr': 0.5, 'n_jobs': 1, 'random_state': 5, 'class_name': 'Selected_SMOTE'}")
INFO:smote_variants:Selected_SMOTE: Running sampling via ('Selected_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'perc_sign_attr': 0.5, 'n_jobs': 1, 'random_state': 5, 'class_name': 'Selected_SMOTE'}")
2022-08-22 19:37:32,349:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
INFO:smote_variants:NearestNeighborsWit

model
RandomForestClassifier()
confusion matrix
[[2120  169]
 [  77 2258]]
classification report
              precision    recall  f1-score   support

         0.0       0.96      0.93      0.95      2289
         1.0       0.93      0.97      0.95      2335

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.946799
Area under the curve : 0.946596


In [40]:
ovs='SMOTE_Cosine'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 19:37:43,353:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 19:37:43,361:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 19:37:43,374:INFO:SMOTE_Cosine: Running sampling via ('SMOTE_Cosine', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': 5, 'class_name': 'SMOTE_Cosine'}")
INFO:smote_variants:SMOTE_Cosine: Running sampling via ('SMOTE_Cosine', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': 5, 'class_name': 'SMOTE_Cosine'}")


model
RandomForestClassifier()
confusion matrix
[[2150  139]
 [ 256 2079]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      0.94      0.92      2289
         1.0       0.94      0.89      0.91      2335

    accuracy                           0.91      4624
   macro avg       0.92      0.91      0.91      4624
weighted avg       0.92      0.91      0.91      4624

Accuracy : 0.914576
Area under the curve : 0.914819


In [42]:
ovs='SMOTE_OUT'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
#OVERSAMPLED DATASET
'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:50:36,349:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:50:36,355:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:50:36,364:INFO:SMOTE_OUT: Running sampling via ('SMOTE_OUT', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'SMOTE_OUT'}")
INFO:smote_variants:SMOTE_OUT: Running sampling via ('SMOTE_OUT', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'random_state': 5, 'class_name': 'SMOTE_OUT'}")
2022-08-22 17:50:36,378:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
INFO:smote_variants:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-08-22 17:50:36,387:INFO

model
RandomForestClassifier()
confusion matrix
[[2180  109]
 [ 145 2190]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.94      2289
         1.0       0.95      0.94      0.95      2335

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.945069
Area under the curve : 0.945141


In [43]:
ovs='SOI_CJ'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
#OVERSAMPLED DATASET
'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:50:40,531:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:50:40,541:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:50:40,551:INFO:SOI_CJ: Running sampling via ('SOI_CJ', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'method': 'jittering', 'n_jobs': 1, 'ss_params': {'n_dim': 1, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'SOI_CJ'}")
INFO:smote_variants:SOI_CJ: Running sampling via ('SOI_CJ', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'method': 'jittering', 'n_jobs': 1, 'ss_params': {'n_dim': 1, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_compo

model
RandomForestClassifier()
confusion matrix
[[2303    0]
 [ 279   12]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      2303
         1.0       1.00      0.04      0.08       291

    accuracy                           0.89      2594
   macro avg       0.95      0.52      0.51      2594
weighted avg       0.90      0.89      0.85      2594

Accuracy : 0.892444
Area under the curve : 0.520619


In [ ]:
ovs='NRSBoundary_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
#OVERSAMPLED DATASET
'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

2022-08-22 17:51:06,378:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
INFO:smote_variants:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-08-22 17:51:06,388:INFO:MulticlassOversampling: Sampling minority class with label: 1.0
INFO:smote_variants:MulticlassOversampling: Sampling minority class with label: 1.0
2022-08-22 17:51:06,399:INFO:NRSBoundary_SMOTE: Running sampling via ('NRSBoundary_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'w': 0.005, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 5, 'class_name': 'NRSBoundary_SMOTE'}")
INFO:smote_variants:NRSBoundary_SMOTE: Running sampling via ('NRSBoundary_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'w': 0.005, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sam

In [ ]:
ovs='SL_graph_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
#OVERSAMPLED DATASET
'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

In [ ]:
ovs='ProWSyn'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
#OVERSAMPLED DATASET
'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

In [ ]:
ovs='SMOTE_RSB'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
#OVERSAMPLED DATASET
'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2295    0]
 [ 294    5]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      2295
         1.0       1.00      0.02      0.03       299

    accuracy                           0.89      2594
   macro avg       0.94      0.51      0.49      2594
weighted avg       0.90      0.89      0.84      2594

Accuracy : 0.886662
Area under the curve : 0.508361


In [ ]:
ovs='SVM_balance'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
#OVERSAMPLED DATASET
'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2193   96]
 [ 163 2146]]
classification report
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.94      2289
         1.0       0.96      0.93      0.94      2309

    accuracy                           0.94      4598
   macro avg       0.94      0.94      0.94      4598
weighted avg       0.94      0.94      0.94      4598

Accuracy : 0.943671
Area under the curve : 0.943733


In [ ]:
ovs='TRIM_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
#OVERSAMPLED DATASET
'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2280    0]
 [ 272 2072]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      2280
         1.0       1.00      0.88      0.94      2344

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.941176
Area under the curve : 0.941980


In [ ]:
ovs='SMOBD'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
#OVERSAMPLED DATASET
'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2066  214]
 [ 257 2087]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90      2280
         1.0       0.91      0.89      0.90      2344

    accuracy                           0.90      4624
   macro avg       0.90      0.90      0.90      4624
weighted avg       0.90      0.90      0.90      4624

Accuracy : 0.898140
Area under the curve : 0.898249


In [ ]:
ovs='DE_oversampling'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
#OVERSAMPLED DATASET
'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2119  193]
 [ 102 2206]]
classification report
              precision    recall  f1-score   support

         0.0       0.95      0.92      0.93      2312
         1.0       0.92      0.96      0.94      2308

    accuracy                           0.94      4620
   macro avg       0.94      0.94      0.94      4620
weighted avg       0.94      0.94      0.94      4620

Accuracy : 0.936147
Area under the curve : 0.936164


In [ ]:
ovs='MSMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)


from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2171  109]
 [ 157 2187]]
classification report
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94      2280
         1.0       0.95      0.93      0.94      2344

    accuracy                           0.94      4624
   macro avg       0.94      0.94      0.94      4624
weighted avg       0.94      0.94      0.94      4624

Accuracy : 0.942474
Area under the curve : 0.942607


In [ ]:
ovs='Safe_Level_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[1933  347]
 [ 340 2004]]
classification report
              precision    recall  f1-score   support

         0.0       0.85      0.85      0.85      2280
         1.0       0.85      0.85      0.85      2344

    accuracy                           0.85      4624
   macro avg       0.85      0.85      0.85      4624
weighted avg       0.85      0.85      0.85      4624

Accuracy : 0.851427
Area under the curve : 0.851378


In [ ]:
ovs='ADOMS'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2161  119]
 [ 159 2185]]
classification report
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94      2280
         1.0       0.95      0.93      0.94      2344

    accuracy                           0.94      4624
   macro avg       0.94      0.94      0.94      4624
weighted avg       0.94      0.94      0.94      4624

Accuracy : 0.939879
Area under the curve : 0.939987


In [ ]:
ovs='SMMO'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2280    0]
 [ 260 2084]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95      2280
         1.0       1.00      0.89      0.94      2344

    accuracy                           0.94      4624
   macro avg       0.95      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.943772
Area under the curve : 0.944539


In [ ]:
ovs='LLE_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2279    1]
 [ 282 2062]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      2280
         1.0       1.00      0.88      0.94      2344

    accuracy                           0.94      4624
   macro avg       0.94      0.94      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.938798
Area under the curve : 0.939627


In [ ]:
ovs='ADASYN'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2163  117]
 [ 153 2191]]
classification report
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94      2280
         1.0       0.95      0.93      0.94      2344

    accuracy                           0.94      4624
   macro avg       0.94      0.94      0.94      4624
weighted avg       0.94      0.94      0.94      4624

Accuracy : 0.941609
Area under the curve : 0.941706


In [ ]:
ovs='Borderline_SMOTE2'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2230   50]
 [ 155 2189]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96      2280
         1.0       0.98      0.93      0.96      2344

    accuracy                           0.96      4624
   macro avg       0.96      0.96      0.96      4624
weighted avg       0.96      0.96      0.96      4624

Accuracy : 0.955666
Area under the curve : 0.955972


In [ ]:
ovs='Borderline_SMOTE1'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2208   72]
 [ 142 2202]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.97      0.95      2280
         1.0       0.97      0.94      0.95      2344

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.953720
Area under the curve : 0.953920


In [ ]:
ovs='SMOTE_ENN'

y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2295    0]
 [ 297    2]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      2295
         1.0       1.00      0.01      0.01       299

    accuracy                           0.89      2594
   macro avg       0.94      0.50      0.48      2594
weighted avg       0.90      0.89      0.83      2594

Accuracy : 0.885505
Area under the curve : 0.503344


In [ ]:
ovs='SMOTE_TomekLinks'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2195  112]
 [ 169 2146]]
classification report
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94      2307
         1.0       0.95      0.93      0.94      2315

    accuracy                           0.94      4622
   macro avg       0.94      0.94      0.94      4622
weighted avg       0.94      0.94      0.94      4622

Accuracy : 0.939204
Area under the curve : 0.939225


In [ ]:
ovs='SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2163  117]
 [ 128 2216]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.95      2280
         1.0       0.95      0.95      0.95      2344

    accuracy                           0.95      4624
   macro avg       0.95      0.95      0.95      4624
weighted avg       0.95      0.95      0.95      4624

Accuracy : 0.947016
Area under the curve : 0.947038


In [ ]:
ovs='MSMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2172  108]
 [ 155 2189]]
classification report
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94      2280
         1.0       0.95      0.93      0.94      2344

    accuracy                           0.94      4624
   macro avg       0.94      0.94      0.94      4624
weighted avg       0.94      0.94      0.94      4624

Accuracy : 0.943123
Area under the curve : 0.943253


In [ ]:
ovs='ANS'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2137  143]
 [ 163 2181]]
classification report
              precision    recall  f1-score   support

         0.0       0.93      0.94      0.93      2280
         1.0       0.94      0.93      0.93      2344

    accuracy                           0.93      4624
   macro avg       0.93      0.93      0.93      4624
weighted avg       0.93      0.93      0.93      4624

Accuracy : 0.933824
Area under the curve : 0.933871


In [ ]:
ovs='distance_SMOTE'
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
oversampler= sv.MulticlassOversampling(oversampler=ovs,
                                        oversampler_params={'random_state': 5})
X_samp, y_samp= oversampler.sample(X, y)
xtrain, xtest, ytrain, ytest = train_test_split(X_samp, y_samp, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2257   23]
 [ 234 2110]]
classification report
              precision    recall  f1-score   support

         0.0       0.91      0.99      0.95      2280
         1.0       0.99      0.90      0.94      2344

    accuracy                           0.94      4624
   macro avg       0.95      0.95      0.94      4624
weighted avg       0.95      0.94      0.94      4624

Accuracy : 0.944420
Area under the curve : 0.945041


# Imbalanced Dataset

In [ ]:
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.30, random_state=0)

In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2295    0]
 [ 295    4]]
classification report
              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94      2295
         1.0       1.00      0.01      0.03       299

    accuracy                           0.89      2594
   macro avg       0.94      0.51      0.48      2594
weighted avg       0.90      0.89      0.83      2594

Accuracy : 0.886276
Area under the curve : 0.506689


# Oversampled Dataset
Conclusions (Results are based on RF classifier):

Oversampled ADASYN SOHE dataset gave best results with oversampling parameters: k=7, ratio=6

Oversampled ADASYN Scaled CatBoost Encoded dataset gave similar results with oversampling parameters: k=8, ratio=6

## Oversampling with SMOTE

In [ ]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Catboost Encoded

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE 
y = delirium_dataset_catboost_encoded["Delirium"].values
X=delirium_dataset_catboost_encoded.drop(["Delirium"], axis=1)
sm = SMOTE(random_state=42,sampling_strategy={1:7000})
X_res, y_res = sm.fit_resample(X, y)
print('Resampled dataset shape {}'.format(Counter(y_res)))
'''Split the resampled data into train & test data with 70:30 mix'''

xtrain, xtest, ytrain, ytest = train_test_split(X_res, y_res, test_size=0.30, random_state=0)
print('xtrain shape')
print(xtrain.shape)
print('xtest shape')
print(xtest.shape)

Resampled dataset shape Counter({0: 7706, 1: 7000})
xtrain shape
(10294, 73)
xtest shape
(4412, 73)


In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2242   61]
 [ 175 1934]]
classification report
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      2303
           1       0.97      0.92      0.94      2109

    accuracy                           0.95      4412
   macro avg       0.95      0.95      0.95      4412
weighted avg       0.95      0.95      0.95      4412

Accuracy : 0.946510
Area under the curve : 0.945268


### Catboost Encoded and Scaled

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE 
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
sm = SMOTE(random_state=42,sampling_strategy={1:7000})
X_res, y_res = sm.fit_resample(X, y)
print('Resampled dataset shape {}'.format(Counter(y_res)))
'''Split the resampled data into train & test data with 70:30 mix'''

xtrain, xtest, ytrain, ytest = train_test_split(X_res, y_res, test_size=0.30, random_state=0)
print('xtrain shape')
print(xtrain.shape)
print('xtest shape')
print(xtest.shape)

Resampled dataset shape Counter({0.0: 7706, 1.0: 7000})
xtrain shape
(10294, 73)
xtest shape
(4412, 73)


In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2219   84]
 [ 153 1956]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.96      0.95      2303
         1.0       0.96      0.93      0.94      2109

    accuracy                           0.95      4412
   macro avg       0.95      0.95      0.95      4412
weighted avg       0.95      0.95      0.95      4412

Accuracy : 0.946283
Area under the curve : 0.945490


### SOHE

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE 
y = delirium_dataset_sohe_encoded["Delirium"].values
X=delirium_dataset_sohe_encoded.drop(["Delirium"], axis=1)
sm = SMOTE(random_state=42,sampling_strategy={1:7000})
X_res, y_res = sm.fit_resample(X, y)
print('Resampled dataset shape {}'.format(Counter(y_res)))
'''Split the resampled data into train & test data with 70:30 mix'''

xtrain, xtest, ytrain, ytest = train_test_split(X_res, y_res, test_size=0.30, random_state=0)
print('xtrain shape')
print(xtrain.shape)
print('xtest shape')
print(xtest.shape)

Resampled dataset shape Counter({0.0: 7706, 1.0: 7000})
xtrain shape
(10294, 156)
xtest shape
(4412, 156)


In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2233   70]
 [ 146 1963]]
classification report
              precision    recall  f1-score   support

         0.0       0.94      0.97      0.95      2303
         1.0       0.97      0.93      0.95      2109

    accuracy                           0.95      4412
   macro avg       0.95      0.95      0.95      4412
weighted avg       0.95      0.95      0.95      4412

Accuracy : 0.951043
Area under the curve : 0.950189


## Oversampling with ADASYN

In [ ]:
!pip install git+https://github.com/stavskal/ADASYN    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/stavskal/ADASYN to /tmp/pip-req-build-je_mb60w
  Running command git clone -q https://github.com/stavskal/ADASYN /tmp/pip-req-build-je_mb60w


In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
from __future__ import unicode_literals

import warnings
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import check_array, check_random_state
from collections import Counter

__author__ = 'stavrianos'

# Link to paper:  bit.ly/22KgAnP


class ADASYN(object):
    """
    Oversampling parent class with the main methods required by scikit-learn:
    fit, transform and fit_transform
    """

    def __init__(self,
                 ratio=0.5,
                 imb_threshold=0.5,
                 k=5,
                 random_state=None,
                 verbose=True):
        """
        :ratio:
            Growth percentage with respect to initial minority
            class size. For example if ratio=0.65 then after
            resampling minority class(es) will have 1.65 times
            its initial size
        :imb_threshold:
            The imbalance ratio threshold to allow/deny oversampling.
            For example if imb_threshold=0.5 then minority class needs
            to be at most half the size of the majority in order for
            resampling to apply
        :k:
            Number of K-nearest-neighbors
        :random_state:
            seed for random number generation
        :verbose:
            Determines if messages will be printed to terminal or not
        Extra Instance variables:
        :self.X:
            Feature matrix to be oversampled
        :self.y:
            Class labels for data
        :self.clstats:
            Class populations to determine minority/majority
        :self.unique_classes_:
            Number of unique classes
        :self.maj_class_:
            Label of majority class
        :self.random_state_:
            Seed
        """

        self.ratio = ratio
        self.imb_threshold = imb_threshold
        self.k = k
        self.random_state = random_state
        self.verbose = verbose
        self.clstats = {}
        self.num_new = 0
        self.index_new = []


    def fit(self, X, y):
        """
        Class method to define class populations and store them as instance
        variables. Also stores majority class label
        """
        self.X = check_array(X)
        self.y = np.array(y).astype(np.int64)
        self.random_state_ = check_random_state(self.random_state)
        self.unique_classes_ = set(self.y)

        # Initialize all class populations with zero
        for element in self.unique_classes_:
            self.clstats[element] = 0

        # Count occurences of each class
        for element in self.y:
            self.clstats[element] += 1

        # Find majority class
        v = list(self.clstats.values())
        k = list(self.clstats.keys())
        self.maj_class_ = k[v.index(max(v))]

        if self.verbose:
            print(
                'Majority class is %s and total number of classes is %s'
                % (self.maj_class_, len(self.unique_classes_)))

    def transform(self, X, y):
        """
        Applies oversampling transformation to data as proposed by
        the ADASYN algorithm. Returns oversampled X,y
        """
        self.new_X, self.new_y = self.oversample()

    def fit_transform(self, X, y):
        """
        Fits the data and then returns the transformed version
        """
        self.fit(X, y)
        self.new_X, self.new_y = self.oversample()

        self.new_X = np.concatenate((self.new_X, self.X), axis=0)
        self.new_y = np.concatenate((self.new_y, self.y), axis=0)

        return self.new_X, self.new_y

    def generate_samples(self, x, knns, knnLabels, cl):

        # List to store synthetically generated samples and their labels
        new_data = []
        new_labels = []
        for ind, elem in enumerate(x):
            # calculating k-neighbors that belong to minority (their indexes in x)
            # Unfortunately knn returns the example itself as a neighbor. So it needs
            # to be ignored thats why it is iterated [1:-1] and
            # knnLabelsp[ind][+1].
            min_knns = [ele for index,ele in enumerate(knns[ind][1:-1])
                         if knnLabels[ind][index +1] == cl]

            if not min_knns:
                continue

            # generate gi synthetic examples for every minority example
            for i in range(0, int(self.gi[ind])):
                # randi holds an integer to choose a random minority kNNs
                randi = self.random_state_.random_integers(
                    0, len(min_knns) - 1)
                # l is a random number in [0,1)
                l = self.random_state_.random_sample()
                # X[min_knns[randi]] is the Xzi on equation [5]
                si = self.X[elem] + \
                    (self.X[min_knns[randi]] - self.X[elem]) * l
                    
                new_data.append(si)
                new_labels.append(self.y[elem])
                self.num_new += 1

        return(np.asarray(new_data), np.asarray(new_labels))

    def oversample(self):
        """
        Preliminary calculations before generation of
        synthetic samples. Calculates and stores as instance
        variables: img_degree(d),G,ri,gi as defined by equations
        [1],[2],[3],[4] in the original paper
        """

        try:
            # Checking if variable exists, i.e. if fit() was called
            self.unique_classes_ = self.unique_classes_
        except:
            raise RuntimeError("You need to fit() before applying tranform(),"
                               "or simply fit_transform()")
        int_X = np.zeros([1, self.X.shape[1]])
        int_y = np.zeros([1])
        # Iterating through all minority classes to determine
        # if they should be oversampled and to what extent
        for cl in self.unique_classes_:
            # Calculate imbalance degree and compare to threshold
            imb_degree = float(self.clstats[cl]) / \
                self.clstats[self.maj_class_]
            if imb_degree > self.imb_threshold:
                if self.verbose:
                    print('Class %s is within imbalance threshold' % cl)
            else:
                # G is the number of synthetic examples to be synthetically
                # produced for the current minority class
                self.G = (self.clstats[self.maj_class_] - self.clstats[cl]) \
                            * self.ratio

                # ADASYN is built upon eucliden distance so p=2 default
                self.nearest_neighbors_ = NearestNeighbors(n_neighbors=self.k + 1)
                self.nearest_neighbors_.fit(self.X)

                # keeping indexes of minority examples
                minx = [ind for ind, exam in enumerate(self.X) if self.y[ind] == cl]

                # Computing kNearestNeighbors for every minority example
                knn = self.nearest_neighbors_.kneighbors(
                    self.X[minx], return_distance=False)

                # Getting labels of k-neighbors of each example to determine how many of them
                # are of different class than the one being oversampled
                knnLabels = self.y[knn.ravel()].reshape(knn.shape)

                tempdi = [Counter(i) for i in knnLabels]

                # Calculating ri as defined in ADASYN paper:
                # No. of k-neighbors belonging to different class than the minority divided by K
                # which is ratio of friendly/non-friendly neighbors
                self.ri = np.array(
                    [(sum(i.values())- i[cl]) / float(self.k) for i in tempdi])

                # Normalizing so that ri is a density distribution (i.e.
                # sum(ri)=1)
                if np.sum(self.ri):
                    self.ri = self.ri / np.sum(self.ri)

                # Calculating #synthetic_examples that need to be generated for
                # each minority instance and rounding to nearest integer because
                # it can't produce e.g 2.35 new examples.
                self.gi = np.rint(self.ri * self.G)

                # Generation of synthetic samples
                inter_X, inter_y = self.generate_samples(
                                     minx, knn, knnLabels, cl)
                # in case no samples where generated at all concatenation
                # won't be attempted
                if len(inter_X):
                    int_X = np.concatenate((int_X, inter_X), axis=0)
                if len(inter_y):
                    int_y = np.concatenate((int_y, inter_y), axis=0)
        # New samples are concatenated in the beggining of the X,y arrays
        # index_new contains the indiced of artificial examples
        self.index_new = [i for i in range(0,self.num_new)]
        return(int_X[1:-1], int_y[1:-1])

In [ ]:
#from adasyn import ADASYN
adsn = ADASYN(k=7,imb_threshold=0.6, ratio=0.75)
X_adasyn, y_adasyn = adsn.fit_transform(X,y)  # your imbalanced dataset is in X,y

Majority class is 0 and total number of classes is 2
Class 0 is within imbalance threshold


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 2 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 0 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead


In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X_adasyn, y_adasyn, test_size=0.30, random_state=0)

### SOHE

In [ ]:
#from adasyn import ADASYN
adsn = ADASYN(k=7,imb_threshold=0.65, ratio=6)
y = delirium_dataset_sohe_encoded["Delirium"].values
X=delirium_dataset_sohe_encoded.drop(["Delirium"], axis=1)
X_adasyn, y_adasyn = adsn.fit_transform(X,y)  # your imbalanced dataset is in X,y
xtrain, xtest, ytrain, ytest = train_test_split(X_adasyn, y_adasyn, test_size=0.30, random_state=0)

Majority class is 0 and total number of classes is 2
Class 0 is within imbalance threshold


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 0 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 2 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 4 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 5 + 1) instead


In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2241  101]
 [ 100 6884]]
classification report
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96      2342
         1.0       0.99      0.99      0.99      6984

    accuracy                           0.98      9326
   macro avg       0.97      0.97      0.97      9326
weighted avg       0.98      0.98      0.98      9326

Accuracy : 0.978447
Area under the curve : 0.971278


In [ ]:
#from adasyn import ADASYN
adsn = ADASYN(k=5,imb_threshold=0.65, ratio=6)
X_adasyn, y_adasyn = adsn.fit_transform(X,y)  # your imbalanced dataset is in X,y
xtrain, xtest, ytrain, ytest = train_test_split(X_adasyn, y_adasyn, test_size=0.30, random_state=0)

In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2261   53]
 [ 129 5222]]
classification report
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.96      2314
         1.0       0.99      0.98      0.98      5351

    accuracy                           0.98      7665
   macro avg       0.97      0.98      0.97      7665
weighted avg       0.98      0.98      0.98      7665

Accuracy : 0.976256
Area under the curve : 0.976494


### OHE

In [ ]:
#from adasyn import ADASYN
adsn = ADASYN(k=5,imb_threshold=0.65, ratio=6)
X_adasyn, y_adasyn = adsn.fit_transform(X,y)  # your imbalanced dataset is in X,y
xtrain, xtest, ytrain, ytest = train_test_split(X_adasyn, y_adasyn, test_size=0.30, random_state=0)

In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2303    3]
 [ 250 4155]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95      2306
         1.0       1.00      0.94      0.97      4405

    accuracy                           0.96      6711
   macro avg       0.95      0.97      0.96      6711
weighted avg       0.97      0.96      0.96      6711

Accuracy : 0.962301
Area under the curve : 0.970973


In [ ]:
#from adasyn import ADASYN
adsn = ADASYN(k=7,imb_threshold=0.65, ratio=3)
X_adasyn, y_adasyn = adsn.fit_transform(X,y)  # your imbalanced dataset is in X,y
xtrain, xtest, ytrain, ytest = train_test_split(X_adasyn, y_adasyn, test_size=0.30, random_state=0)

In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2345    4]
 [ 244 2916]]
classification report
              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95      2349
         1.0       1.00      0.92      0.96      3160

    accuracy                           0.95      5509
   macro avg       0.95      0.96      0.95      5509
weighted avg       0.96      0.95      0.96      5509

Accuracy : 0.954983
Area under the curve : 0.960541


In [ ]:
#from adasyn import ADASYN
adsn = ADASYN(k=7,imb_threshold=0.6, ratio=0.75)
X_adasyn, y_adasyn = adsn.fit_transform(X,y)  # your imbalanced dataset is in X,y
xtrain, xtest, ytrain, ytest = train_test_split(X_adasyn, y_adasyn, test_size=0.30, random_state=0)

In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2329    1]
 [ 249  706]]
classification report
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95      2330
         1.0       1.00      0.74      0.85       955

    accuracy                           0.92      3285
   macro avg       0.95      0.87      0.90      3285
weighted avg       0.93      0.92      0.92      3285

Accuracy : 0.923896
Area under the curve : 0.869419


### CatBoost Encoded Data

In [ ]:
#from adasyn import ADASYN
adsn = ADASYN(k=5,imb_threshold=0.65, ratio=6)
X_adasyn, y_adasyn = adsn.fit_transform(X,y)  # your imbalanced dataset is in X,y
xtrain, xtest, ytrain, ytest = train_test_split(X_adasyn, y_adasyn, test_size=0.30, random_state=0)

Majority class is 0 and total number of classes is 2
Class 0 is within imbalance threshold


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 0 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 2 + 1) instead


In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2284   22]
 [ 163 4242]]
classification report
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96      2306
         1.0       0.99      0.96      0.98      4405

    accuracy                           0.97      6711
   macro avg       0.96      0.98      0.97      6711
weighted avg       0.97      0.97      0.97      6711

Accuracy : 0.972433
Area under the curve : 0.976728


In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler().fit(delirium_dataset_catboost_encoded)
X_train_copy = delirium_dataset_catboost_encoded.copy(deep=True)
X_train_copy = mms.transform(delirium_dataset_catboost_encoded)
X_train_copy

array([[0.39204349, 0.78842347, 0.06863552, ..., 0.15338738, 0.28728858,
        0.02709164],
       [0.40714202, 0.78861565, 0.0701923 , ..., 0.14380473, 0.29660596,
        0.01894226],
       [0.39359442, 0.78858091, 0.06378386, ..., 0.16384517, 0.27521449,
        0.03745735],
       ...,
       [0.3639504 , 0.79474853, 0.0683293 , ..., 0.16455777, 0.28324311,
        0.01965856],
       [0.34426382, 0.79243819, 0.06746426, ..., 0.16649929, 0.28573266,
        0.02047151],
       [0.34785279, 0.79418629, 0.07168317, ..., 0.16603911, 0.28415928,
        0.02224521]])

In [ ]:
df=pd.DataFrame(X_train_copy, columns =list(delirium_dataset_catboost_encoded))

In [ ]:
df.to_csv("delirium_dataset_catboost_encoded_and_scaled.csv")

### Catboost Encoded and Scaled

In [ ]:
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.30, random_state=0)
#from adasyn import ADASYN
adsn = ADASYN(k=8,imb_threshold=0.65, ratio=6)
X_adasyn, y_adasyn = adsn.fit_transform(X,y)  # your imbalanced dataset is in X,y
xtrain, xtest, ytrain, ytest = train_test_split(X_adasyn, y_adasyn, test_size=0.30, random_state=0)

Majority class is 0 and total number of classes is 2
Class 0 is within imbalance threshold


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 2 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 0 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 5 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 4 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarni

In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2169  131]
 [  96 6701]]
classification report
              precision    recall  f1-score   support

         0.0       0.96      0.94      0.95      2300
         1.0       0.98      0.99      0.98      6797

    accuracy                           0.98      9097
   macro avg       0.97      0.96      0.97      9097
weighted avg       0.97      0.98      0.97      9097

Accuracy : 0.975047
Area under the curve : 0.964460


In [ ]:
y = df["Delirium"].values
X=df.drop(["Delirium"], axis=1)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.30, random_state=0)
#from adasyn import ADASYN
adsn = ADASYN(k=5,imb_threshold=0.65, ratio=6)
X_adasyn, y_adasyn = adsn.fit_transform(X,y)  # your imbalanced dataset is in X,y
xtrain, xtest, ytrain, ytest = train_test_split(X_adasyn, y_adasyn, test_size=0.30, random_state=0)

Majority class is 0 and total number of classes is 2
Class 0 is within imbalance threshold


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 0 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 2 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead


In [ ]:
#OVERSAMPLED DATASET

'''Random Forest Classifier on resampled data'''

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rfmodel = RandomForestClassifier()
rfmodel.fit(xtrain,ytrain)
print('model')
print(rfmodel)

ypredrf = rfmodel.predict(xtest)
print('confusion matrix')
print(metrics.confusion_matrix(ytest, ypredrf))
print('classification report')
print(metrics.classification_report(ytest, ypredrf))
print('Accuracy : %f' % (metrics.accuracy_score(ytest, ypredrf)))
print('Area under the curve : %f' % (metrics.roc_auc_score(ytest, ypredrf)))

model
RandomForestClassifier()
confusion matrix
[[2207   56]
 [ 128 4605]]
classification report
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.96      2263
         1.0       0.99      0.97      0.98      4733

    accuracy                           0.97      6996
   macro avg       0.97      0.97      0.97      6996
weighted avg       0.97      0.97      0.97      6996

Accuracy : 0.973699
Area under the curve : 0.974105


# Deep Learning Model for Delirium Prediction

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
#from adasyn import ADASYN
adsn = ADASYN(k=7,imb_threshold=0.65, ratio=6)
y = delirium_dataset_sohe_encoded["Delirium"].values
X=delirium_dataset_sohe_encoded.drop(["Delirium"], axis=1)
X_adasyn, y_adasyn = adsn.fit_transform(X,y)  # your imbalanced dataset is in X,y
xtr, xtest, ytr, ytest = train_test_split(X_adasyn, y_adasyn, test_size=0.30, random_state=0)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtr, ytr, test_size=0.20, random_state=0)

Majority class is 0 and total number of classes is 2
Class 0 is within imbalance threshold


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 0 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: DeprecationWarning: This function is deprecated. Please call randint(0, 2 + 1) instead


In [ ]:
# make y labels that work nicely with Keras
y_train_keras = to_categorical(ytrain, 2)
y_valid_keras = to_categorical(yvalid,2)
y_test_keras = to_categorical(ytest,2)
y_train_keras.shape

(17407, 2)